<a href="https://colab.research.google.com/github/NeskaCleo/MonicaGlez/blob/main/PySpark/Ejercicio_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### `Práctica Spark`

**Vamos a trabajar con un dataset compuesto de tres ficheros. Estos tres ficheros contienen las transacciones de venta, los productos y los clientes que participaron en dichas transacciones**


## Instalacion de Java

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Instalacion de Spark


In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
#Obtener las versiones de spark la pagina web
url = 'https://downloads.apache.org/spark/'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)

In [ ]:
# leer la pagina web y obtener las versiones de spark disponibles
link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x]
print(spark_link)

['spark-3.1.3/', 'spark-3.2.2/', 'spark-3.3.0/']


In [ ]:
ver_spark = spark_link[1][:-1] # obtener la version y eliminar el caracter '/' del final
print(ver_spark)

spark-3.2.2


In [ ]:
import os # libreria de manejo del sistema operativo
#instalar automaticamente la version deseadda de spark
!wget -q https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop2.7.tgz
!tar xf spark-3.2.2-bin-hadoop2.7.tgz
# instalar pyspark
#!pip install -q pyspark

In [ ]:
!pip install -q pyspark==3.2.2

# Cargar pyspark en el sistema

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, types as T
spark = SparkSession.builder.appName("Varios").master("local[*]").getOrCreate()
spark

## Definir variables de entorno

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.2-bin-hadoop2.7"

Tenemos 3 datasets en esta practica:

-Transactions: que contiene la información relativa a órdenes de compra

-Productos

-Clientes

Une todos los datasets en uno solo, teniendo en cuenta que vamos a analizar la información según orden de compra, producto y cliente. Cada registro (fila) es una entrada de la orden de compra que está asociado a la venta de un product id. La información del cliente se puede repetir en los diferentes registros que componen la orden de compra

Resuelve las siguientes preguntas:
1. Monta el dataset para resolver adecuadamente los ejercicios abajo descritos. (1 pto)
2. Devuelve los siguientes indicadores generando una nueva columna para cada uno. Nota: resolver con usando withColumn y UDF en dos de las preguntas, pudiendo usar SQL en la restante (2 pts):
a. Precio de venta por unidad (teniendo en cuenta el descuento aplicado)
b. Beneficio unitario (calculado como beneficio entre cantidad)
c. Coste Unitario: calculado como precio de venta unitario - beneficio unitario
3.Desde control de gestión, dicen que hay que intentar ser rentables. Identifica la rentabilidad obtenida por cada categoria de producto (profit por categoria de producto) (1 pto).
4. Nos piden también que identifiquemos los productos a los que se le han aplicado descuento y han obtenido un beneficio negativo en la transacción (1 pto)
5. Identifica a los clientes a los que se vendieron esos productos (los identificados en la pregunta 4) y calcula su rentabilidad media (1 pto)
6. Si quisiéramos obtener un beneficio unitario de un 50% sobre el coste unitario en estos productos, ¿cuál habría sido el precio de venta unitario con y sin descuento del 10%? (2 pts)
7. El presidente Johnny Walker se ha levantado generoso por su cumpleaños. Quiere regalar sendos (dos!) vales para un viaje al Caribe al cliente más rentable y al cliente más fiel (medido en número de transacciones) de Florida (2 pto).

# Importación de datos

**Une todos los datasets en uno solo, teniendo en cuenta que vamos a analizar la información según orden de compra, producto y cliente. Cada registro (fila) es una entrada de la orden de compra que está asociado a la venta de un product id. La información del cliente se puede repetir en los diferentes registros que componen la orden de compra**

In [ ]:
transaciones=spark.read.csv('/content/drive/MyDrive/us_superstore_transactions.csv',inferSchema=True, header=True, sep=',')
transaciones.show(10)

+------+--------------+----------+----------+--------------+-----------+---------------+------------------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Product ID|             Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+------------------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|FUR-BO-10001798|            261.96|       2|     0.0| 41.9136|
|     2|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|FUR-CH-10000454|            731.94|       3|     0.0| 219.582|
|     3|CA-2016-138688| 6/12/2016| 6/16/2016|  Second Class|   DV-13045|OFF-LA-10000240|             14.62|       2|     0.0|  6.8714|
|     4|US-2015-108966|10/11/2015|10/18/2015|Standard Class|   SO-20335|FUR-TA-10000577|          957.5775|       5|    0.45|-383.031|
|     5|US-2015-108966|10/11/2015|10/18/2015|Standard C

In [ ]:
transaciones.describe().show()

+-------+------------------+--------------+----------+---------+--------------+-----------+---------------+-----------------+-----------------+-------------------+------------------+
|summary|            Row ID|      Order ID|Order Date|Ship Date|     Ship Mode|Customer ID|     Product ID|            Sales|         Quantity|           Discount|            Profit|
+-------+------------------+--------------+----------+---------+--------------+-----------+---------------+-----------------+-----------------+-------------------+------------------+
|  count|              9994|          9994|      9994|     9994|          9994|       9994|           9994|             9994|             9994|               9994|              9994|
|   mean|            4997.5|          null|      null|     null|          null|       null|           null|229.8580008304938|3.789573744246548|0.15620272163298934|28.656896307784802|
| stddev|2885.1636290974325|          null|      null|     null|          null|      

In [ ]:
transaciones = transaciones.drop('Row ID','Order ID','Order Date','Ship Date','Ship Mode')

In [ ]:
transaciones.describe().show()

+-------+-----------+---------------+-----------------+-----------------+-------------------+------------------+
|summary|Customer ID|     Product ID|            Sales|         Quantity|           Discount|            Profit|
+-------+-----------+---------------+-----------------+-----------------+-------------------+------------------+
|  count|       9994|           9994|             9994|             9994|               9994|              9994|
|   mean|       null|           null|229.8580008304938|3.789573744246548|0.15620272163298934|28.656896307784802|
| stddev|       null|           null|623.2451005086809|2.225109691141402|0.20645196782571626| 234.2601076909573|
|    min|   AA-10315|FUR-BO-10000112|            0.444|                1|                0.0|         -6599.978|
|    max|   ZD-21925|TEC-PH-10004977|         22638.48|               14|                0.8|          8399.976|
+-------+-----------+---------------+-----------------+-----------------+-------------------+---

In [ ]:
clientes=spark.read.csv('/content/drive/MyDrive/us_superstores_customers.csv',inferSchema=True, header=True, sep=';')
clientes.show(10)

+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+
|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|
+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+
|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|
|   DV-13045|   Darrin Van Huff|  Corporate|United States|    Los Angeles|    California|      90036|   West|
|   SO-20335|    Sean O'Donnell|   Consumer|United States|Fort Lauderdale|       Florida|      33311|  South|
|   BH-11710|   Brosina Hoffman|   Consumer|United States|    Los Angeles|    California|      90032|   West|
|   AA-10480|      Andrew Allen|   Consumer|United States|        Concord|North Carolina|      28027|  South|
|   IM-15070|      Irene Maddox|   Consumer|United States|        Seattle|    Washington|      98103|   West|
|   HP-148

In [ ]:
clientes.describe().show()

+-------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+
|summary|Customer ID|     Customer Name|    Segment|      Country|    City|  State|       Postal Code| Region|
+-------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+
|  count|       4910|              4910|       4910|         4910|    4910|   4910|              4910|   4910|
|   mean|       null|              null|       null|         null|    null|   null|55345.687576374745|   null|
| stddev|       null|              null|       null|         null|    null|   null|31922.029402318818|   null|
|    min|   AA-10315|     Aaron Bergman|   Consumer|United States|Aberdeen|Alabama|              1040|Central|
|    max|   ZD-21925|Zuschuss Donatelli|Home Office|United States|    Yuma|Wyoming|             99301|   West|
+-------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+



In [ ]:
clientes = clientes.dropDuplicates(['Customer ID'])

In [ ]:
clientes = clientes.drop('Segment','City','Region')

In [ ]:
clientes.describe().show()

+-------+-----------+------------------+-------------+---------+------------------+
|summary|Customer ID|     Customer Name|      Country|    State|       Postal Code|
+-------+-----------+------------------+-------------+---------+------------------+
|  count|        793|               793|          793|      793|               793|
|   mean|       null|              null|         null|     null|55229.717528373265|
| stddev|       null|              null|         null|     null|31679.223815801197|
|    min|   AA-10315|     Aaron Bergman|United States|  Alabama|              1841|
|    max|   ZD-21925|Zuschuss Donatelli|United States|Wisconsin|             99207|
+-------+-----------+------------------+-------------+---------+------------------+



In [ ]:
productos=spark.read.csv('/content/drive/MyDrive/us_superstores_products.csv',inferSchema=True, header=True, sep=';')
productos.show(10)

+---------------+---------------+------------+--------------------+
|     Product ID|       Category|Sub-Category|        Product Name|
+---------------+---------------+------------+--------------------+
|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|
|FUR-CH-10000454|      Furniture|      Chairs|Hon Deluxe Fabric...|
|OFF-LA-10000240|Office Supplies|      Labels|Self-Adhesive Add...|
|FUR-TA-10000577|      Furniture|      Tables|Bretford CR4500 S...|
|OFF-ST-10000760|Office Supplies|     Storage|Eldon Fold 'N Rol...|
|FUR-FU-10001487|      Furniture| Furnishings|Eldon Expressions...|
|OFF-AR-10002833|Office Supplies|         Art|          Newell 322|
|TEC-PH-10002275|     Technology|      Phones|Mitel 5320 IP Pho...|
|OFF-BI-10003910|Office Supplies|     Binders|DXL Angle-View Bi...|
|OFF-AP-10002892|Office Supplies|  Appliances|Belkin F5C206VTEL...|
+---------------+---------------+------------+--------------------+
only showing top 10 rows



In [ ]:
productos.describe().show()

+-------+---------------+----------+------------+--------------------+
|summary|     Product ID|  Category|Sub-Category|        Product Name|
+-------+---------------+----------+------------+--------------------+
|  count|           1894|      1894|        1894|                1894|
|   mean|           null|      null|        null|                null|
| stddev|           null|      null|        null|                null|
|    min|FUR-BO-10000112| Furniture| Accessories|"""While you Were...|
|    max|TEC-PH-10004977|Technology|      Tables|netTALK DUO VoIP ...|
+-------+---------------+----------+------------+--------------------+



In [ ]:
productos = productos.dropDuplicates(['Product ID'])

In [ ]:
productos = productos.drop('Product Name')

In [ ]:
productos.describe().show()

+-------+---------------+----------+------------+
|summary|     Product ID|  Category|Sub-Category|
+-------+---------------+----------+------------+
|  count|           1862|      1862|        1862|
|   mean|           null|      null|        null|
| stddev|           null|      null|        null|
|    min|FUR-BO-10000112| Furniture| Accessories|
|    max|TEC-PH-10004977|Technology|      Tables|
+-------+---------------+----------+------------+



In [ ]:
resultadointermedio = clientes.select('Customer ID','Customer Name','Country','State','Postal Code').join(transaciones,on='Customer ID',how='left')
resultadointermedio.show(10)

+-----------+-------------+-------------+---------+-----------+---------------+------------------+--------+--------+---------+
|Customer ID|Customer Name|      Country|    State|Postal Code|     Product ID|             Sales|Quantity|Discount|   Profit|
+-----------+-------------+-------------+---------+-----------+---------------+------------------+--------+--------+---------+
|   AA-10315|   Alex Avila|United States|Minnesota|      55407|FUR-FU-10002456|             14.56|       2|     0.0|   6.2608|
|   AA-10315|   Alex Avila|United States|Minnesota|      55407|OFF-PA-10000349|             14.94|       3|     0.0|   7.0218|
|   AA-10315|   Alex Avila|United States|Minnesota|      55407|TEC-AC-10002857|             41.72|       7|     0.2|   5.7365|
|   AA-10315|   Alex Avila|United States|Minnesota|      55407|TEC-PH-10000895|           431.976|       3|     0.2|  32.3982|
|   AA-10315|   Alex Avila|United States|Minnesota|      55407|OFF-FA-10001332|2.3040000000000003|       1|    

In [ ]:
resultadointermedio.describe().show()

+-------+-----------+------------------+-------------+---------+------------------+---------------+------------------+-----------------+-------------------+------------------+
|summary|Customer ID|     Customer Name|      Country|    State|       Postal Code|     Product ID|             Sales|         Quantity|           Discount|            Profit|
+-------+-----------+------------------+-------------+---------+------------------+---------------+------------------+-----------------+-------------------+------------------+
|  count|       9994|              9994|         9994|     9994|              9994|           9994|              9994|             9994|               9994|              9994|
|   mean|       null|              null|         null|     null| 55320.68060836502|           null|229.85800083049372|3.789573744246548|0.15620272163298943|28.656896307784624|
| stddev|       null|              null|         null|     null|31471.270959004716|           null| 623.2451005086804|  

In [ ]:
midata = resultadointermedio.select('Customer ID','Customer Name','Country','Postal Code','State','Product ID','Sales','Quantity','Discount','Profit').join(productos,on='Product ID',how='left')
midata.show(10)

+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+---------+---------------+------------+
|     Product ID|Customer ID|Customer Name|      Country|Postal Code|    State|             Sales|Quantity|Discount|   Profit|       Category|Sub-Category|
+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+---------+---------------+------------+
|FUR-FU-10002456|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.56|       2|     0.0|   6.2608|      Furniture| Furnishings|
|OFF-PA-10000349|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.94|       3|     0.0|   7.0218|Office Supplies|       Paper|
|TEC-AC-10002857|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             41.72|       7|     0.2|   5.7365|     Technology| Accessories|
|TEC-PH-10000895|   AA-10315|   Alex Avila|United States|      5

In [ ]:
midata.describe().show()

+-------+---------------+-----------+------------------+-------------+------------------+---------+------------------+-----------------+-------------------+------------------+----------+------------+
|summary|     Product ID|Customer ID|     Customer Name|      Country|       Postal Code|    State|             Sales|         Quantity|           Discount|            Profit|  Category|Sub-Category|
+-------+---------------+-----------+------------------+-------------+------------------+---------+------------------+-----------------+-------------------+------------------+----------+------------+
|  count|           9994|       9994|              9994|         9994|              9994|     9994|              9994|             9994|               9994|              9994|      9994|        9994|
|   mean|           null|       null|              null|         null| 55320.68060836502|     null|229.85800083049372|3.789573744246548|0.15620272163298943|28.656896307784624|      null|        null|


**Devuelve los siguientes indicadores generando una nueva columna para cada uno. Nota: resolver con usando withColumn y UDF en dos de las preguntas, pudiendo usar SQL en la restante**

**a. Precio de venta por unidad (teniendo en cuenta el descuento aplicado)**

In [ ]:
from pyspark.sql.functions import lit, udf, col,array
from pyspark.sql.types import *

In [ ]:
@udf(returnType=FloatType())
def unidad (a,b,c):
  x=(a-a*c)/b
  return x
midata2=midata.withColumn('Precio_unidad',unidad('Sales','Quantity','Discount'))
midata2.show(10)


+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+---------+---------------+------------+-------------+
|     Product ID|Customer ID|Customer Name|      Country|Postal Code|    State|             Sales|Quantity|Discount|   Profit|       Category|Sub-Category|Precio_unidad|
+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+---------+---------------+------------+-------------+
|FUR-FU-10002456|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.56|       2|     0.0|   6.2608|      Furniture| Furnishings|         7.28|
|OFF-PA-10000349|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.94|       3|     0.0|   7.0218|Office Supplies|       Paper|         4.98|
|TEC-AC-10002857|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             41.72|       7|     0.2|   5.7365|     Technology| Accesso

**b. Beneficio unitario (calculado como beneficio entre cantidad)**

In [ ]:
@udf(returnType=FloatType())
def bunidad (j,k):
  x=j/k
  return x
midata3=midata2.withColumn('Beneficio_unidad',bunidad('Profit','Quantity'))
midata3.show(10)

+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+---------+---------------+------------+-------------+----------------+
|     Product ID|Customer ID|Customer Name|      Country|Postal Code|    State|             Sales|Quantity|Discount|   Profit|       Category|Sub-Category|Precio_unidad|Beneficio_unidad|
+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+---------+---------------+------------+-------------+----------------+
|FUR-FU-10002456|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.56|       2|     0.0|   6.2608|      Furniture| Furnishings|         7.28|          3.1304|
|OFF-PA-10000349|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.94|       3|     0.0|   7.0218|Office Supplies|       Paper|         4.98|          2.3406|
|TEC-AC-10002857|   AA-10315|   Alex Avila|United States|      55

**c. Coste Unitario: calculado como precio de venta unitario - beneficio unitario**

In [ ]:
midata4=midata3.withColumn('Coste_unidad',F.format_number(F.col('Precio_unidad')-F.col('Beneficio_unidad'),2))
midata4.show()

+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+-----------------+---------------+------------+-------------+----------------+------------+
|     Product ID|Customer ID|Customer Name|      Country|Postal Code|    State|             Sales|Quantity|Discount|           Profit|       Category|Sub-Category|Precio_unidad|Beneficio_unidad|Coste_unidad|
+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+-----------------+---------------+------------+-------------+----------------+------------+
|FUR-FU-10002456|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.56|       2|     0.0|           6.2608|      Furniture| Furnishings|         7.28|          3.1304|        4.15|
|OFF-PA-10000349|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.94|       3|     0.0|           7.0218|Office Supplies|       Paper|     

**3.Desde control de gestión, dicen que hay que intentar ser rentables. Identifica la rentabilidad obtenida por cada categoria de producto (profit por categoria de producto) (1 pto)**

In [ ]:
midata.groupby('Category').sum('Profit').show()

+---------------+------------------+
|       Category|       sum(Profit)|
+---------------+------------------+
|Office Supplies|       122490.8008|
|      Furniture|18451.272799999973|
|     Technology|145454.94809999972|
+---------------+------------------+



Y podemos calcular también la media

In [ ]:
midata.groupby('Category').mean('Profit').show()

+---------------+-----------------+
|       Category|      avg(Profit)|
+---------------+-----------------+
|Office Supplies| 20.3270495851311|
|      Furniture| 8.69932710985383|
|     Technology|78.75200221981576|
+---------------+-----------------+



**4. Nos piden también que identifiquemos los productos a los que se le han aplicado descuento y han obtenido un beneficio negativo en la transacción (1 pto)**

In [ ]:
midata1 = midata.where((F.col('Profit')<0) & ( midata.Discount>0)).select('Customer Name','Category','Sub-Category','Profit','Discount').sort(F.col('Profit').asc())
midata1.show(10)

+----------------+---------------+------------+----------+--------+
|   Customer Name|       Category|Sub-Category|    Profit|Discount|
+----------------+---------------+------------+----------+--------+
|   Cindy Stewart|     Technology|    Machines| -6599.978|     0.7|
|  Grant Thornton|     Technology|    Machines|-3839.9904|     0.5|
|     Luke Foster|Office Supplies|     Binders|-3701.8928|     0.8|
|  Sharelle Roach|     Technology|    Machines|  -3399.98|     0.7|
|   Henry Goldwyn|Office Supplies|     Binders|-2929.4845|     0.8|
|Natalie Fritzler|     Technology|    Machines|-2639.9912|     0.7|
|     Nathan Cano|Office Supplies|     Binders| -2287.782|     0.8|
|     Joseph Holt|      Furniture|      Tables|-1862.3124|     0.4|
|Zuschuss Carroll|Office Supplies|     Binders|-1850.9464|     0.8|
|     Sean Miller|     Technology|    Machines|-1811.0784|     0.5|
+----------------+---------------+------------+----------+--------+
only showing top 10 rows



In [ ]:
midata1 = midata.where((F.col('Profit')<0) & ( midata.Discount>0)).select('Customer Name','Category','Sub-Category','Profit','Discount').sort(F.col('Discount').desc())
midata1.show()

+------------------+---------------+------------+-------------------+--------+
|     Customer Name|       Category|Sub-Category|             Profit|Discount|
+------------------+---------------+------------+-------------------+--------+
|       Dean Braden|Office Supplies|     Binders|-21.168000000000003|     0.8|
|     Sibella Parks|Office Supplies|  Appliances|          -126.8592|     0.8|
|        Emily Phan|Office Supplies|     Binders|           -68.1856|     0.8|
|     Harold Pawlan|Office Supplies|     Binders|-3.8160000000000003|     0.8|
|         Dean Katz|Office Supplies|     Binders|            -17.046|     0.8|
|      Rick Bensley|Office Supplies|     Binders|            -3.0396|     0.8|
|Christina Anderson|Office Supplies|     Binders|            -14.773|     0.8|
|  Jennifer Braxton|Office Supplies|  Appliances|            -131.12|     0.8|
| Eugene Hildebrand|Office Supplies|     Binders|-3.4619999999999997|     0.8|
|      Logan Currie|Office Supplies|  Appliances|   

**5. Identifica a los clientes a los que se vendieron esos productos (los identificados en la pregunta 4) y calcula su rentabilidad media (1 pto)**

In [ ]:
midata1.groupby('Customer Name').agg(F.mean('Profit').alias ('Media_profit')).sort(F.col('Media_profit').asc()).show()


+----------------+-------------------+
|   Customer Name|       Media_profit|
+----------------+-------------------+
|   Cindy Stewart|          -3452.185|
|  Grant Thornton|         -2093.5539|
|     Joseph Holt|         -1862.3124|
|   Tracy Hopkins|         -1306.5504|
|    David Bremer|          -803.0835|
|     Jim Radford| -786.7439999999999|
|      Alex Avila|          -786.0144|
|     Nathan Cano| -780.8848666666668|
|  Sharelle Roach|         -693.42516|
|   Henry Goldwyn| -599.7669000000001|
|   Karen Daniels| -582.3095666666667|
|     Daniel Lacy|          -571.9956|
|Natalie Fritzler| -566.6939199999999|
|     Luke Foster| -543.6498571428572|
|Maureen Fritzler|          -459.9875|
|        John Lee| -456.5733333333333|
|     Sean Miller|           -415.392|
|      Philip Fox|-407.68199999999996|
|    Paul Knutson|          -403.8885|
|     Olvera Toch| -402.7673333333334|
+----------------+-------------------+
only showing top 20 rows



In [ ]:
midata1.groupby('Customer Name','Category').agg(F.mean('Profit').alias ('Media_profit')).sort(F.col('Media_profit').asc()).show(25)

+----------------+---------------+-------------------+
|   Customer Name|       Category|       Media_profit|
+----------------+---------------+-------------------+
|   Cindy Stewart|     Technology|          -6599.978|
|  Grant Thornton|     Technology|         -3839.9904|
|  Sharelle Roach|     Technology|           -3399.98|
|     Joseph Holt|      Furniture|         -1862.3124|
|    Becky Martin|     Technology|-1359.9920000000002|
|Natalie Fritzler|     Technology|         -1328.4946|
|   Tracy Hopkins|Office Supplies|         -1306.5504|
|     Nathan Cano|Office Supplies|         -1148.1274|
|    David Bremer|Office Supplies|           -1141.47|
|     Luke Foster|Office Supplies| -938.5393000000001|
|  Laurel Beltran|     Technology|            -938.28|
|     Sean Miller|     Technology|          -913.7522|
|     Olvera Toch|Office Supplies|           -913.176|
| Tracy Blumstein|      Furniture|          -893.9646|
|   Tanja Norvell|      Furniture|          -814.4832|
|     Jim 

**6. Si quisiéramos obtener un beneficio unitario de un 50% sobre el coste unitario en estos productos, ¿cuál habría sido el precio de venta unitario con y sin descuento del 10%? (2 pts)**

### **Con 0% descuento**

In [ ]:
midata4.show(5)

+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+-------+---------------+------------+-------------+----------------+------------+
|     Product ID|Customer ID|Customer Name|      Country|Postal Code|    State|             Sales|Quantity|Discount| Profit|       Category|Sub-Category|Precio_unidad|Beneficio_unidad|Coste_unidad|
+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+--------+-------+---------------+------------+-------------+----------------+------------+
|FUR-FU-10002456|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.56|       2|     0.0| 6.2608|      Furniture| Furnishings|         7.28|          3.1304|        4.15|
|OFF-PA-10000349|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.94|       3|     0.0| 7.0218|Office Supplies|       Paper|         4.98|          2.3406|        2.64|
|TEC-AC-10

In [ ]:
midata5 = midata4.select('*',lit('0.1').alias('Descuento')).drop('Discount')
midata5.show()

+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+-----------------+---------------+------------+-------------+----------------+------------+---------+
|     Product ID|Customer ID|Customer Name|      Country|Postal Code|    State|             Sales|Quantity|           Profit|       Category|Sub-Category|Precio_unidad|Beneficio_unidad|Coste_unidad|Descuento|
+---------------+-----------+-------------+-------------+-----------+---------+------------------+--------+-----------------+---------------+------------+-------------+----------------+------------+---------+
|FUR-FU-10002456|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.56|       2|           6.2608|      Furniture| Furnishings|         7.28|          3.1304|        4.15|      0.1|
|OFF-PA-10000349|   AA-10315|   Alex Avila|United States|      55407|Minnesota|             14.94|       3|           7.0218|Office Supplies|       Paper|         4

In [ ]:
midata6=midata5.withColumn(('Precio_unidad_0'),F.col('Coste_unidad')*1.5).select('Coste_unidad','Precio_unidad_0')
midata6.show(10)

+------------+------------------+
|Coste_unidad|   Precio_unidad_0|
+------------+------------------+
|      114.80|             172.2|
|      299.43|           449.145|
|      314.06|471.09000000000003|
|        1.66|2.4899999999999998|
|      169.19|           253.785|
|       19.75|            29.625|
|        1.29|             1.935|
|        0.55|0.8250000000000001|
|        2.35|3.5250000000000004|
|       28.03|            42.045|
+------------+------------------+
only showing top 10 rows



En este ejercicio tengo muchas dudas ya que

precio unitario = (ventas - (ventas*descuento))/cantidad

beneficio unitario= beneficio/cantidad (ahora fijo = 0.5coste unitario)

coste unitario =precio unitario - beneficio unitario

coste unitario = precio unitario - 0.5coste unitario -> precio unitario = 1.5 coste unitario

Pero es que el coste unitario que tenemos ya calculado incluye el descuento anterior, así que si lo calculo cómo he hecho incluye el descuento del dataframe primero. También lo he calculado sobre el original, donde ya había productos con 0% de descuento, en estos el precio unitario sale menor que el original claro.




### **Con 10% descuento**

In [ ]:
midata_df=midata6.withColumn(('Precio_unidad_0.1'),F.col('Precio_unidad_0')-(F.col('Precio_unidad_0')*0.1))
midata_df.show()

+------------+------------------+------------------+
|Coste_unidad|   Precio_unidad_0| Precio_unidad_0.1|
+------------+------------------+------------------+
|      114.80|             172.2|            154.98|
|      299.43|           449.145|          404.2305|
|      314.06|471.09000000000003|           423.981|
|        1.66|2.4899999999999998|2.2409999999999997|
|      169.19|           253.785|          228.4065|
|       19.75|            29.625|           26.6625|
|        1.29|             1.935|            1.7415|
|        0.55|0.8250000000000001|            0.7425|
|        2.35|3.5250000000000004|3.1725000000000003|
|       28.03|            42.045|           37.8405|
|       45.39| 68.08500000000001|61.276500000000006|
|        1.25|             1.875|            1.6875|
|        1.33|             1.995|            1.7955|
|       33.59|50.385000000000005|45.346500000000006|
|      145.20|217.79999999999998|196.01999999999998|
|       17.50|             26.25|            2

**7. El presidente Johnny Walker se ha levantado generoso por su cumpleaños. Quiere regalar sendos (dos!) vales para un viaje al Caribe al cliente más rentable y al cliente más fiel (medido en número de transacciones) de Florida (2 pto).**

In [ ]:
midata7= midata.groupBy('Customer ID','State').sum('Profit')
midata7.show(10)

+-----------+--------------+-------------------+
|Customer ID|         State|        sum(Profit)|
+-----------+--------------+-------------------+
|   AA-10315|     Minnesota|-362.88250000000016|
|   AA-10375|       Arizona|           277.3824|
|   AA-10480|North Carolina| 435.82739999999995|
|   AA-10645|  Pennsylvania|  857.8033000000001|
|   AB-10015|    Washington|           129.3465|
|   AB-10060|      New York|          2054.5885|
|   AB-10105|       Arizona| 5444.8054999999995|
|   AB-10150|      New York| 313.65969999999993|
|   AB-10165|    California| 220.81300000000002|
|   AB-10255|          Ohio| 264.56750000000005|
+-----------+--------------+-------------------+
only showing top 10 rows



In [ ]:
midata8 = midata7.filter((midata7.State == 'Florida')).select('Customer ID', F.col('State').alias('Estado'), F.col('sum(Profit)').alias('Más rentable')).sort(F.col('Más rentable').desc())
midata8.show()

+-----------+-------+-------------------+
|Customer ID| Estado|       Más rentable|
+-----------+-------+-------------------+
|   AJ-10795|Florida|          1158.7064|
|   SC-20695|Florida|           611.8291|
|   AP-10915|Florida|           579.3113|
|   SL-20155|Florida|  527.9715000000001|
|   DB-12910|Florida|           431.3748|
|   BW-11200|Florida| 246.99729999999997|
|   ES-14080|Florida| 246.66610000000006|
|   MK-18160|Florida|           227.8338|
|   RD-19720|Florida|           207.3304|
|   CA-12775|Florida| 204.48950000000002|
|   KN-16450|Florida|           114.3083|
|   DB-13555|Florida|            109.327|
|   JR-15700|Florida|            107.985|
|   HJ-14875|Florida|            98.6548|
|   PH-18790|Florida|            47.8878|
|   AS-10240|Florida| 33.715799999999994|
|   LW-16825|Florida|            32.5767|
|   RH-19555|Florida|0.30570000000000075|
|   MG-18205|Florida|            -1.2453|
|   DO-13435|Florida| -38.90860000000001|
+-----------+-------+-------------

### **Ya lo tenemos: El cliente más rentable es**

### *   Customer ID = AJ-10795
### *   Beneficio = 1158.7064





In [ ]:
midata9= midata.groupBy('Customer ID','State').count()
midata9.show(10)

+-----------+--------------+-----+
|Customer ID|         State|count|
+-----------+--------------+-----+
|   AA-10315|     Minnesota|   11|
|   AA-10375|       Arizona|   15|
|   AA-10480|North Carolina|   12|
|   AA-10645|  Pennsylvania|   18|
|   AB-10015|    Washington|    6|
|   AB-10060|      New York|   18|
|   AB-10105|       Arizona|   20|
|   AB-10150|      New York|   12|
|   AB-10165|    California|   14|
|   AB-10255|          Ohio|   14|
+-----------+--------------+-----+
only showing top 10 rows



In [ ]:
midata10 = midata9.filter((midata9.State == 'Florida')).select('Customer ID', F.col('State').alias('Estado'), F.col('count').alias('Transaciones')).sort(F.col('Transaciones').desc())
midata10.show()

+-----------+-------+------------+
|Customer ID| Estado|Transaciones|
+-----------+-------+------------+
|   AP-10915|Florida|          31|
|   AJ-10795|Florida|          21|
|   ES-14080|Florida|          20|
|   RD-19720|Florida|          20|
|   CA-12775|Florida|          17|
|   LF-17185|Florida|          16|
|   LW-16825|Florida|          15|
|   SO-20335|Florida|          15|
|   DB-13555|Florida|          14|
|   SC-20695|Florida|          14|
|   DO-13435|Florida|          13|
|   AS-10240|Florida|          13|
|   DB-12910|Florida|          12|
|   MK-18160|Florida|          11|
|   BW-11200|Florida|          10|
|   AG-10525|Florida|           9|
|   SL-20155|Florida|           9|
|   AH-10195|Florida|           8|
|   KN-16450|Florida|           8|
|   HJ-14875|Florida|           6|
+-----------+-------+------------+
only showing top 20 rows



### **Ya lo tenemos: El cliente más fiel es**

### *   Customer ID = AP-10915
### *   Transaciones = 31
